In [1]:
import os
import numpy as np
import networkx as nx
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)




In [2]:
# Load a single graph from a .edges file
def load_graph(edges_file):
    G = nx.read_edgelist(edges_file, create_using=nx.Graph(), nodetype=int)
    return G

# Load multiple graphs from a folder containing .edges files
def load_graphs_from_folder(folder_path):
    graphs = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".edges"):
            file_path = os.path.join(folder_path, file_name)
            G = load_graph(file_path)
            graphs.append((G, file_name))
    return graphs



In [3]:
# Initialize network state with all nodes uninfected, except one initial infected node
def initialize_state(G):
    state = {node: 'uninfected' for node in G.nodes}
    initial_infected = random.choice(list(G.nodes))
    state[initial_infected] = 'infected'
    return state, initial_infected

# Calculate static propagation probability for SRPM
def calculate_static_propagation_prob(u, v, G, alpha=0.5, beta=0.5):
    out_degree_u = max(G.degree(u), 1)
    in_degree_v = max(G.degree(v), 1)
    return (alpha * np.log(1 + out_degree_u)) / ((alpha * np.log(1 + out_degree_u)) + (beta * np.log(1 + in_degree_v)))

# Calculate dynamic propagation probability for DRPM
def calculate_dynamic_propagation_prob(u, v, G, alpha_t):
    return alpha_t * calculate_static_propagation_prob(u, v, G)



In [4]:
# Define DQN model for RL agent
class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, action_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)



In [5]:
# Define RL agent with experience replay
class RumorInfluenceAgent:
    def __init__(self, state_size, action_size, gamma=0.99, epsilon=1.0, epsilon_min=0.1, epsilon_decay=0.995):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.model = DQN(state_size, action_size)
        self.target_model = DQN(state_size, action_size)
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        self.update_target_network()

    def update_target_network(self):
        self.target_model.load_state_dict(self.model.state_dict())

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        with torch.no_grad():
            return torch.argmax(self.model(torch.from_numpy(state).float())).item()

    def replay(self, batch_size=64):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target += self.gamma * torch.max(self.target_model(torch.from_numpy(next_state).float())).item()
            target_f = self.model(torch.from_numpy(state).float())
            target_f[action] = target
            self.optimizer.zero_grad()
            loss = nn.MSELoss()(target_f, torch.tensor(target))
            loss.backward()
            self.optimizer.step()
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay



In [6]:
def create_node_mapping(G):
    nodes = list(G.nodes)
    node_to_index = {node: idx for idx, node in enumerate(nodes)}
    index_to_node = {idx: node for idx, node in enumerate(nodes)}
    return node_to_index, index_to_node

In [7]:
# Simulate rumor spread in the network
def spread_rumor(G, state, propagation_model, alpha_t=None):
    new_state = state.copy()
    for node in state:
        if state[node] == 'infected':
            for neighbor in G.neighbors(node):
                if state[neighbor] == 'uninfected':
                    if propagation_model == 'static':
                        prob = calculate_static_propagation_prob(node, neighbor, G)
                    elif propagation_model == 'dynamic' and alpha_t:
                        prob = calculate_dynamic_propagation_prob(node, neighbor, G, alpha_t)
                    if np.random.rand() <= prob:
                        new_state[neighbor] = 'infected'
    return new_state



In [8]:
def take_action(state, action_node, G, blocker_budget):
    next_state = state.copy()
    reward = 0
    if next_state[action_node] == 'uninfected' and blocker_budget > 0:
        next_state[action_node] = 'blocked'
        reward += 1

    next_state = spread_rumor(G, next_state, propagation_model='dynamic', alpha_t=0.5)
    infected_count = sum(1 for node in next_state if next_state[node] == 'infected')
    reward -= infected_count
    done = infected_count == 0 or sum(1 for node in next_state if next_state[node] == 'uninfected') == 0
    return next_state, reward, done

In [9]:
def train_agent(G, agent, node_to_index, index_to_node, epochs=2, blocker_budget=1):
    for epoch in range(epochs):
        state, initial_infected = initialize_state(G)
        total_reward = 0
        done = False
        while not done:
            # Convert state to array representation suitable for agent
            state_array = np.array([1 if state[index_to_node[idx]] == 'infected' else 0 for idx in range(len(G.nodes))])
            action = agent.act(state_array)
            action_node = index_to_node[action]  # Convert action index to actual node
            next_state, reward, done = take_action(state, action_node, G, blocker_budget)
            agent.remember(state_array, action, reward, state_array, done)
            state = next_state
            total_reward += reward
            if len(agent.memory) > 64:
                agent.replay()
        agent.update_target_network()
        print(f"Epoch: {epoch}, Total Reward: {total_reward}")

In [10]:
# Evaluate agent on a single graph
def evaluate_agent(G, agent, initial_state, epochs=2):
    success_rate = 0
    infection_rates = []
    for epoch in range(epochs):
        state = initial_state.copy()
        total_infected = 0
        done = False
        while not done:
            action = agent.act(state)
            next_state, reward, done = take_action(state, action, G, blocker_budget=1)
            state = next_state
            total_infected += sum(1 for node in state if state[node] == 'infected')
        infection_rate = total_infected / len(G.nodes)
        infection_rates.append(infection_rate)
        if infection_rate < 0.1:
            success_rate += 1
    average_infection_rate = np.mean(infection_rates)
    success_rate /= epochs
    print(f"Average Infection Rate: {average_infection_rate:.4f}, Success Rate: {success_rate:.2f}")
    return average_infection_rate, success_rate



In [11]:
def train_and_evaluate_on_all_graphs(folder_path):
    graphs = load_graphs_from_folder(folder_path)
    for G, file_name in graphs:
        print(f"Training on graph from file: {file_name}")
        state, initial_infected = initialize_state(G)
        node_to_index, index_to_node = create_node_mapping(G)  # Create the mappings
        agent = RumorInfluenceAgent(state_size=len(G.nodes), action_size=len(G.nodes))

        # Train the agent on this graph
        train_agent(G, agent, node_to_index, index_to_node)

        # Evaluate the trained model
        print(f"Evaluating on graph from file: {file_name}")
        evaluate_agent(G, agent, state, node_to_index, index_to_node)

In [12]:
# Main function to run training and evaluation
if __name__ == "__main__":
    dataset_folder = "dataset"  # Replace with actual path
    train_and_evaluate_on_all_graphs(dataset_folder)

Training on graph from file: 0.edges


c:\Users\rjaga\anaconda3\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([333])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


KeyboardInterrupt: 